In [1]:
from helper import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
#Define Children Calculator

In [3]:
#we use add function and mystery function as a functions
#define children calculator

from llama_index.core.tools import FunctionTool

def add(x: int, y: int) -> int:
    """Adds two integers together."""
    return x + y

def mystery(x: int, y: int) -> int: 
    """Mystery function that operates on top of two numbers."""
    return (x + y) * (x + y)


add_tool = FunctionTool.from_defaults(fn=add) #calling to add function
mystery_tool = FunctionTool.from_defaults(fn=mystery) #calling to mystery function

In [4]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo")
response = llm.predict_and_call(
    [add_tool, mystery_tool], 
    "Tell me the output of the mystery function on 2 and 9", #only mension mystery function
    verbose=True
)
print(str(response))

=== Calling Function ===
Calling function: mystery with args: {"x": 2, "y": 9}
=== Function Output ===
121
121


In [ ]:
#Define an Auto-Retrieval Tool

In [5]:
#Load the data
from llama_index.core import SimpleDirectoryReader

#Load the document
documents = SimpleDirectoryReader(input_files=["Design of a cooling system for a hybrid-powered vehicle.pdf"]).load_data()

In [6]:
#split the documents to chunks
from llama_index.core.node_parser import SentenceSplitter
splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [9]:
print(nodes[0].get_content(metadata_mode="all")) #nodes[0] mean labeled page data(1st page)

"""
metadata--> page_label: 1
    file_name: Design of a cooling system for a hybrid-powered vehicle.pdf
    file_path: Design of a cooling system for a hybrid-powered vehicle.pdf
    file_type: application/pdf
    file_size: 701257
    creation_date: 2024-10-20
    last_modified_date: 2024-10-20
"""

page_label: 1
file_name: Design of a cooling system for a hybrid-powered vehicle.pdf
file_path: Design of a cooling system for a hybrid-powered vehicle.pdf
file_type: application/pdf
file_size: 701257
creation_date: 2024-10-20
last_modified_date: 2024-10-20

Al-Mustaqbal Journal of Sustainability in Engineering Sciences Al-Mustaqbal Journal of Sustainability in Engineering Sciences 
Volume 1 Issue 2 Article 5 
2023 
Design of a cooling system for a hybrid-power ed vehicle Design of a cooling system for a hybrid-power ed vehicle 
Qusa y Rasheed Al- Amir 
Air Conditioning and Refriger ation T echniques Engineering Depar tment, Al-Mustaqbal Univ ersity , Iraq, 
dr.qusa yrasheed24@mustaqbal-college.edu.iq 
Follow this and additional works at: https:/ /ajses.uomus.edu.iq/home 
Recommended Citation Recommended Citation 
Amir , Qusa y Rasheed Al- (2023) "Design of a cooling system for a hybrid-power ed vehicle, " Al-Mustaqbal 
Journal of Sustainability in Engineering Sciences : Vol. 1 : Iss. 

'\nmetadata--> page_label: 1\n    file_name: Design of a cooling system for a hybrid-powered vehicle.pdf\n    file_path: Design of a cooling system for a hybrid-powered vehicle.pdf\n    file_type: application/pdf\n    file_size: 701257\n    creation_date: 2024-10-20\n    last_modified_date: 2024-10-20\n'

In [10]:
from llama_index.core import VectorStoreIndex #this will basically build a RAG indexing pipeline over these nodes. and it will add an embedding for each node and it will get back a query engine.

vector_index = VectorStoreIndex(nodes)
query_engine = vector_index.as_query_engine(similarity_top_k=2)

In [11]:
#METADATA Filtering

from llama_index.core.vector_stores import MetadataFilters #we can retrive data using filtering metada

query_engine = vector_index.as_query_engine(
    similarity_top_k=2,
    filters=MetadataFilters.from_dicts(
        [
            {"key": "page_label", "value": "2"} #filtering data using page label
        ]
    )
)

response = query_engine.query(
    "How to design a cooling system ?", 
)

In [12]:
print(str(response))

To design a cooling system for a hybrid-powered vehicle, one can utilize a combination of kinetic energy from exhaust gas recycling and solar energy from a photovoltaic system. The kinetic energy generated from waste exhaust gases can be converted into electrical energy by passing it through a turbine connected to a dynamo when the vehicle is in motion. Additionally, when the vehicle is stationary, solar energy can be harnessed by connecting photovoltaic cells to the battery, storing and converting it into electrical energy to operate the compressor. This hybrid cooling system reduces engine load and fuel consumption, thereby increasing engine efficiency. By implementing these methods, the cooling system can operate efficiently both during vehicle operation and when the engine is off, leading to improved overall performance and reduced energy waste.


In [13]:
#print out the metadata attached to these source nodes.we can get it for create ai plegarism ditector tool
for n in response.source_nodes:
    print(n.metadata)
    
    #this response mean page label 2 has this informations

{'page_label': '2', 'file_name': 'Design of a cooling system for a hybrid-powered vehicle.pdf', 'file_path': 'Design of a cooling system for a hybrid-powered vehicle.pdf', 'file_type': 'application/pdf', 'file_size': 701257, 'creation_date': '2024-10-20', 'last_modified_date': '2024-10-20'}


In [ ]:
#Define Auto Retrival Tool

In [14]:
from typing import List
from llama_index.core.vector_stores import FilterCondition


def vector_query(
    query: str, 
    page_numbers: List[str]
) -> str:
    """Perform a vector search over an index.
    
    query (str): the string query to be embedded.
    page_numbers (List[str]): Filter by set of pages. Leave BLANK if we want to perform a vector search
        over all pages. Otherwise, filter by the set of specified pages.
    
    """

    metadata_dicts = [
        {"key": "page_label", "value": p} for p in page_numbers
    ]
    
    query_engine = vector_index.as_query_engine(
        similarity_top_k=2,
        filters=MetadataFilters.from_dicts(
            metadata_dicts,
            condition=FilterCondition.OR
        )
    )
    response = query_engine.query(query)
    return response
    

vector_query_tool = FunctionTool.from_defaults(
    name="vector_tool",
    fn=vector_query
)

In [15]:
llm = OpenAI(model="gpt-3.5-turbo", temperature=0)
response = llm.predict_and_call(
    [vector_query_tool], 
    "How to design a cooling system as described on page 2?", #filterd by page number
    verbose=True
)

=== Calling Function ===
Calling function: vector_tool with args: {"query": "design a cooling system", "page_numbers": ["2"]}
=== Function Output ===
The cooling system designed for the hybrid-powered vehicle utilizes a combination of kinetic energy from exhaust gas recycling and solar energy from a photovoltaic system. This hybrid energy is converted into electrical energy to power the electric compressor of the refrigeration system. When the vehicle is in motion, the kinetic energy from waste exhaust gases is converted into electrical energy through a turbine connected to a dynamo. Conversely, when the vehicle is stationary, solar energy is used to operate the cooling system by connecting photovoltaic cells to the battery. This innovative design significantly reduces engine load and fuel consumption, leading to increased engine efficiency. The study demonstrated that the cooling system can operate both during vehicle operation and when the engine is off, resulting in improved engine 

In [16]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '2', 'file_name': 'Design of a cooling system for a hybrid-powered vehicle.pdf', 'file_path': 'Design of a cooling system for a hybrid-powered vehicle.pdf', 'file_type': 'application/pdf', 'file_size': 701257, 'creation_date': '2024-10-20', 'last_modified_date': '2024-10-20'}


In [ ]:
#Adding Summary index and Query Engine Tool

In [18]:
#add 1st code indexes summery index and query enginetool

from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool

summary_index = SummaryIndex(nodes)
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
summary_tool = QueryEngineTool.from_defaults(
    name="summary_tool",
    query_engine=summary_query_engine,
    description=(
        "Useful if you want to get a summary of pdf"
    ),
)

In [19]:
response = llm.predict_and_call(
    [vector_query_tool, summary_tool], 
    "Give me summary of data reduction described on page 32?", 
    verbose=True
)

=== Calling Function ===
Calling function: summary_tool with args: {"input": "data reduction", "page_numbers": ["32"]}
=== Function Output ===
Data reduction techniques are essential for managing and analyzing large datasets efficiently. These methods help simplify complex datasets, remove redundant information, and extract key features for further analysis. By applying data reduction techniques, researchers can improve computational efficiency, reduce storage requirements, and enhance the overall performance of data processing tasks.


In [20]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '1', 'file_name': 'Design of a cooling system for a hybrid-powered vehicle.pdf', 'file_path': 'Design of a cooling system for a hybrid-powered vehicle.pdf', 'file_type': 'application/pdf', 'file_size': 701257, 'creation_date': '2024-10-20', 'last_modified_date': '2024-10-20'}
{'page_label': '2', 'file_name': 'Design of a cooling system for a hybrid-powered vehicle.pdf', 'file_path': 'Design of a cooling system for a hybrid-powered vehicle.pdf', 'file_type': 'application/pdf', 'file_size': 701257, 'creation_date': '2024-10-20', 'last_modified_date': '2024-10-20'}
{'page_label': '3', 'file_name': 'Design of a cooling system for a hybrid-powered vehicle.pdf', 'file_path': 'Design of a cooling system for a hybrid-powered vehicle.pdf', 'file_type': 'application/pdf', 'file_size': 701257, 'creation_date': '2024-10-20', 'last_modified_date': '2024-10-20'}
{'page_label': '3', 'file_name': 'Design of a cooling system for a hybrid-powered vehicle.pdf', 'file_path': 'Design of a co

In [21]:
#calling to function
response = llm.predict_and_call(
    [vector_query_tool, summary_tool], 
    "What is a summary of the paper?", 
    verbose=True
)

=== Calling Function ===
Calling function: summary_tool with args: {"input": "paper"}
=== Function Output ===
The paper discusses the design and implementation of a cooling system for a hybrid-powered vehicle that utilizes a combination of kinetic energy from exhaust gas recycling and solar energy to power the electric compressor of the refrigeration system. It focuses on improving engine efficiency, reducing fuel consumption, and increasing the coefficient of performance compared to conventional cooling systems. Additionally, the paper explores various refrigeration cycles suitable for vehicle cooling applications, emphasizing the importance of clean energy sources and waste heat recovery for environmental preservation and energy efficiency.
